# Trying out `EvolvingGraphs`

We're interested in using the [`EvolvingGraphs`](http://evolvinggraphsjl.readthedocs.io/en/latest/overview.html).

In [5]:
Pkg.add("EvolvingGraphs")
using EvolvingGraphs

INFO: Nothing to be done
